In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import random
import re
import os
import timeit

%matplotlib inline
import matplotlib.pyplot as plt
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

From specifications and descriptions of the dataset we can infer the proper `dtype` for most of the columns:
- numerical : `int`,`float`
- text : `str`
- A low/finite number of values : `categorical`,`boolean`

In [ ]:
class Dataset:
    def __init__(self, file:str):
        self.file_name : str = file
        self.types : dict = self.load_types()
        self.df : pd.DataFrame = None
        
    def name(self):
        return re.search(r'\/([\w\d]*)\.csv',self.file_name).group(1)
    
    def types_file(self):
        return '/kaggle/working/'+ self.name() +'.npy'
        
    def save_types(self):
        np.save(self.types_file(), self.types)
        
    def load_types(self):
        if os.path.isfile(self.types_file()):
            self.types = np.load(self.types_file(),allow_pickle='TRUE').item()
            return self.types
        return None
    
    def __getitem__(self,item):
        return self.df[item]


datasets = [
    Dataset(file='../input/instagram-dataset/instagram_profiles.csv'), 
    Dataset(file='../input/instagram-dataset/instagram_locations.csv'), 
    Dataset(file='../input/instagram-dataset/instagram_posts.csv')
]
profiles, locations, posts = datasets

profiles.types = {
#     'sid' : int,
    'profile_id' : int,
    'profile_name' : pd.StringDtype(storage='pyarrow'),
    'firstname_lastname' : pd.StringDtype(storage='pyarrow'),
    'description' : pd.StringDtype(storage='pyarrow'),
    'following' : int,
    'followers' : int,
    'n_posts' : int,
    'url' : pd.StringDtype(storage='pyarrow'),
#     'cts' : pd.DatetimeTZDtype(tz='UTC'), #obtained through parse_dates=['cts']
    'is_business_account' : pd.BooleanDtype()
}
posts.types = {
#     'sid' : int, # gives an error, maybe because it's the index?
    'sid_profile' : int,
    'post_id' : pd.StringDtype(storage='pyarrow'),
    'profile_id' : int,
    'location_id' : int,
    'description' : pd.StringDtype(storage='pyarrow'),
    'post_type' : pd.CategoricalDtype(categories=[1,2,3]),
    'numbr_likes' : int,
    'number_comments' : int
}
locations.types = {
#     'sid' : int, # gives an error, maybe because it's the index?
    'id' : int,
    'name' : pd.StringDtype(storage='pyarrow'),
    'street' : pd.StringDtype(storage='pyarrow'),
    'zip' : pd.StringDtype(storage='pyarrow'),
    'city' : pd.StringDtype(storage='pyarrow'),
    'region' : pd.StringDtype(storage='pyarrow'),
    'cd' : pd.CategoricalDtype(),
    'phone' : pd.StringDtype(storage='pyarrow'),
    'aj_exact_city_match' : pd.BooleanDtype(),
    'aj_exact_country_match' : pd.BooleanDtype(),
    'blurb' : pd.StringDtype(storage='pyarrow'),
    'dir_city_id' : pd.StringDtype(storage='pyarrow'),
    'dir_city_name' : pd.StringDtype(storage='pyarrow'),
    'dir_city_slug' : pd.StringDtype(storage='pyarrow'),
    'dir_country_id' : pd.CategoricalDtype(),
    'dir_country_name' : pd.CategoricalDtype(),
    'lat' : pd.Float32Dtype(),
    'lng' : pd.Float32Dtype(),
    'primary_alias_on_fb' : pd.StringDtype(storage='pyarrow'),
    'slug' : pd.StringDtype(storage='pyarrow'),
    'website' : pd.StringDtype(storage='pyarrow'),
}

In [ ]:
def get_types(signed=True, unsigned=True, custom=[]):
    '''Returns a pandas dataframe containing the boundaries of each integer dtype'''
    # based on https://stackoverflow.com/a/57894540/9419492
    pd_types = custom
    if signed:
        pd_types += [pd.Int8Dtype() ,pd.Int16Dtype() ,pd.Int32Dtype(), pd.Int64Dtype()]
    if unsigned:
        pd_types += [pd.UInt8Dtype() ,pd.UInt16Dtype(), pd.UInt32Dtype(), pd.UInt64Dtype()]
    type_df = pd.DataFrame(data=pd_types, columns=['pd_type'])
    type_df['np_type'] = type_df['pd_type'].apply(lambda t: t.numpy_dtype)
    type_df['min_value'] = type_df['np_type'].apply(lambda row: np.iinfo(row).min)
    type_df['max_value'] = type_df['np_type'].apply(lambda row: np.iinfo(row).max)
    type_df['allow_negatives'] = type_df['min_value'] < 0
    type_df['size'] = type_df['np_type'].apply(lambda row: row.itemsize)
    type_df.sort_values(by=['size', 'allow_negatives'], inplace=True)
    return type_df.reset_index(drop=True)
    
get_types()

In [ ]:
def downcast_int(file_path, column:str, chunksize=10000, delimiter=',', signed=True, unsigned=True):
    '''Assigns the smallest possible dtype to an integer column of a csv'''
    types = get_types(signed, unsigned)
    negatives = False
    for chunk in pd.read_csv(file_path, usecols=[column],delimiter=delimiter,skiprows=lambda x:x%100==0,chunksize=chunksize):
        M = chunk[column].max()
        m = chunk[column].min()
        if not signed and not negatives and m < 0 :
            types = types[types['allow_negatives']] # removes unsigned rows
            negatives = True
        if m < types['min_value'].iloc[0]:
            types = types[types['min_value'] < m]
        if M > types['max_value'].iloc[0]:
            types = types[types['max_value'] > M]
        if len(types) == 1:
            print('early stop')
            break
    return types['pd_type'].iloc[0]

def optimize_cols(file, int_cols, delimiter=',', signed=True, unsigned=True):
    out = dict()
    for col in int_cols:
        out[col] = downcast_int(file, col, delimiter=delimiter, signed=signed, unsigned=unsigned)
    return out

In [ ]:
for ds in datasets:
    if not ds.load_types():
        int_cols = [k for k,v in ds.types.items() if v is int]
        print(int_cols)
        ds.types.update(optimize_cols(ds.file_name, int_cols, delimiter='\t'))
        print(f'Optimized {len(int_cols)} types for {ds.name()}')
#     ds.types.update({k:pd.StringDtype(storage='pyarrow') for k,v in ds.types.items() if v==pd.StringDtype(storage='pyarrow')})
    ds.save_types()
#     print(ds.types)

In [ ]:
SAMPLE_SIZE = 10000
for ds in datasets:
    ds.df = pd.read_csv(ds.file_name, dtype=ds.types, index_col='sid', delimiter='\t', parse_dates=['cts'], nrows=SAMPLE_SIZE)
    avg_mem_unoptimized = pd.read_csv(ds.file_name, index_col='sid', delimiter='\t', nrows=SAMPLE_SIZE).memory_usage(deep=True).sum()/SAMPLE_SIZE
    avg_mem_optimized = ds.df.memory_usage(deep=True).sum()/SAMPLE_SIZE
    print(f'{ds.name().ljust(19)} mean optimized memory usage per entry:  {round(avg_mem_optimized):3} B vs {round(avg_mem_unoptimized):4} B  : {round(avg_mem_optimized/avg_mem_unoptimized*100,2):5}%') 

In [ ]:
# Load test:  fails even with optimizing can't load everything in 16GB
# for ds in datasets:
#     ds.df = pd.read_csv(ds.file_name, dtype=ds.types, index_col='sid', delimiter='\t', parse_dates=['cts'])
#     print(f'Loaded {ds.name().ljust(19)}, size = {round(ds.df.memory_usage(deep=True).sum()/ 1024**3,2)}GB')
#     del ds.df

# Exploratory Data Analysis [RQ1]

### Profiles 
| Field | Description |
|---|---|
| SID | Sequence ID |
| profile_id | Instagrams ID |
| profile_name | profile name |
| firstname_lastname | firstname lastname |
| description | May contain '\n' |
| following | Number of following profile at the moment it was visited |
| followers | Number of followers at the moment it was visited |
| n_posts | Number of posts at the moment it was visited |
| url | Url in profile description |
| cts | Timestamp when the profile was visited |
| is_business_account | Boolean flag if that profile was makred by the owner as business account |

In [ ]:
profiles.df.head()

In [ ]:
profiles.df[['following', 'followers', 'n_posts']].describe()

In [ ]:
profiles.df.info()

### Locations

| Field | Description |
|---|---|
| SID | Sequence ID |
| ID | Instagrams ID |
| Name | Locations Name |
| Street | Street Address, may contain '\n' |
| ZIP | Zip code |
| City | City Name |
| Region | Region |
| CD | Country Code |
| Phone | The phone in format as on the Instragram |
| aj_exact_city_match | The Instagrams Internal key |
| aj_exact_country_match | The Instagrams Internal key |
| blurb | Description of the place, may contain '\n' |
| dir_city_id | The Instagrams internal City ID |
| dir_city_name | city Name |
| dir_city_slug | City tag (sortof) |
| dir_country_id | Country ID |
| dir_country_name | country |
| lat | Latitude |
| lng | Longtitude |
| primary_alias_on_fb | Bool Flag |
| slug | ??? |
| website | The URL to web site, may contain more then 1 URL, may contain '\n' |
| cts | Timestamp when the location was visited |

In [ ]:
locations.df.iloc[:,:10].head()

In [ ]:
locations.df.iloc[:,10:].head()

In [ ]:
locations.df.describe()
# it's useless

In [ ]:
locations.df.info()

### Posts
| Field | Description |
|---|---|
| SID | Sequence ID |
| sid_profile | Sequence ID of the profile from *Profiles* table |
| post_id | Instagrams ID |
| profile_id | Instagrams ID may be null |
| location_id | Instagrams ID |
| cts | Timestamp when the Post was created |
| post_type | 1 - Photo, 2 - Video, 3 - multy |
| description | May contain '\n' |
| number_likes | Number of Likes at the moment it was visited |
| number_comments | Number of comments at the moment it was visited |

In [ ]:
# posts.columns = posts.columns.str.replace('numbr', 'number')
posts.df.head()

In [ ]:
posts.df[['number_likes','number_comments']].describe()

In [ ]:
posts.df.info()

In [ ]:
# # Chunking 
# for chunk in pandas.read_csv("FILE.csv", chunksize=1000):
#     pass

categorical## Before Starting
We note that this dataset has many odd values, with many columns having missing values that we'll have to deal somehow

Furthermore we need to adjust all those columns having datetime objects not actually being represented as objects

In [ ]:
# done above

In [ ]:
def count_problematic(df):
    at_least_one_null = len(df[pd.isnull(df).any(axis=1)])
    return at_least_one_null, len(df) - at_least_one_null

for i, df in enumerate([posts, locations, profiles]):
    plt.subplot(1,3,i+1)
#     plt.title(name)
    plt.pie(count_problematic(df), labels=['missing values', ''])

# [RQ2] 

#### Plot the number of posts for each profile in descending order.

In [ ]:
profiles.df.sort_values(by='n_posts', ascending=False)[['profile_name', 'n_posts']]

#### What posts have the most number of likes?

In [ ]:
posts.df.sort_values(by='numbr_likes', ascending=False).head(10)

#### What posts have the most and the least number of comments?

In [ ]:
posts.df.sort_values(by='number_comments', ascending=False).head(10)

In [ ]:
posts.df.sort_values(by='number_comments', ascending=True).head(10)

#### How many posts include tagged locations, and how many do not? Show it using an appropriate chart and comment your results.

In [ ]:
null_locations = sum(posts['location_id'].isna())
plt.pie([null_locations, len(posts.df)-null_locations], labels=['null', 'tagged'])

#### How many posts include only photos? How many also have videos?

In [ ]:
def categorize(mapping, values):
    res = pd.DataFrame.from_dict(mapping, orient='index', columns=['count'])
    for k,v in mapping.items():
        res.loc[k] = sum(values == v)
    res.loc['NA'] = sum(values.isna())
    if res.loc['NA'].sum() == 0:
        return res.loc[mapping.keys()]
    else:
        return res

In [ ]:
post_types = posts['post_type'].value_counts(dropna=False)
sns.barplot(y=post_types, x=post_types.index).set(title='Post types')

#### What's the percentage of business accounts vs non-business? What can you interpret regarding that percentage?

In [ ]:
business_accounts = profiles.df['is_business_account'].value_counts(dropna=False)
sns.barplot(x=[True, False, '<NA>'], y=business_accounts).set(title='Business Accounts')